<a href="https://colab.research.google.com/github/microsoft/FLAML/blob/main/notebook/autogen_agent_function_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interative LLM Agent with Function Calls

FLAML offers an experimental feature of interactive LLM agents, which can be used to solve various tasks with human or automatic feedback, including tasks that require using tools via code. We now support the new feature of OpenAI models to make function calls to pre-defined functions. We allow passing in and updating pre-defined arguments to the functions, and we also allow calling to functions of a class instance (More details in `UserProxyAgent`).

In this notebook, we demonstrate how to use `AssistantAgent` and `UserProxyAgent` to make function calls (For details of the two agents please check out `autogen_agent_auto_feedback_from_code_execution.ipynb`). A specified prompt and function configs need to be passed to `AssistantAgent` to initialize the agent. The corresponding functions need to be passed to `UserProxyAgent`, which will be responsible for executing any function calls made by `AssistantAgent`. 

## Requirements

FLAML requires `Python>=3.7`. To run this notebook example, please install flaml with the [mathchat] option since we will use functions from `MathUserProxyAgent`:
```bash
pip install flaml[mathchat]
```

In [1]:
# %pip install flaml[mathchat]==2.0.0rc2

## Set your API Endpoint

Function call are supported only in "gpt-4-0613" or "gpt-3.5-turbo-0613", Store your key in `"key_openai.txt"` and create a config_list with it.

If you open this notebook in google colab, you can upload your files by click the file icon on the left panel and then choose "upload file" icon.


In [1]:
# function call are supported only in "gpt-4-0613" or "gpt-3.5-turbo-0613", we will use "gpt-4-0613" in this example
config_list = config_list = [{
    'model' : 'gpt-4-0613',
    'api_key': open("key_openai.txt").read().strip(),
}]

## Making Function Calls

In this example, we demonstrate function call execution with `AssistantAgent` and `UserProxyAgent`. We use the function that calls Wolfram Alpha API in `MathUserProxyAgent` to be our pre-defined function.


In [2]:
import os
from flaml.autogen.agent import AssistantAgent, UserProxyAgent, MathUserProxyAgent

# you need to provide a wolfram alpha appid to run this example
if not os.environ.get("WOLFRAM_ALPHA_APPID"):
    os.environ["WOLFRAM_ALPHA_APPID"] = open("wolfram.txt").read().strip()


sys_prompt = """You are an advanced AI with the capability to solve complex math problems.
Wolfram alpha is provided as an external service to help you solve math problems.

When the user gives a math problem, please use the most efficient way to solve the problem.
You are encouraged to use Wolfram alpha whenever it is possible during the solving process. For example, simplications, calculations, equation solving, etc.
However, if the operation requires little computation (very simple calculations, etc), you can also solve it directly.
Reply "TERMINATE" in the end when everything is done.
"""
oai_config = {
    "model": "gpt-4-0613",
    "functions": [
        {
            "name": "query_wolfram",
            "description": "Return the API query result from the Wolfram Alpha.",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "The Wolfram Alpha code to be executed.",
                    }
                },
                "required": ["code"],
            },
        }
    ],
    "function_call": "auto",
}
chatbot = AssistantAgent("chatbot", sys_prompt, config_list=config_list, **oai_config)


# function to be passed
def query_wolfram_API(code):
    mathagent = MathUserProxyAgent()
    return mathagent._execute_one_wolfram_query(code)[0]

# the key in `function_map` should match the function name passed to OpenAI
user = UserProxyAgent(
    "user",
    human_input_mode="NEVER",
    function_map={"query_wolfram": {"function": query_wolfram_API}},
)

# start the conversation
chatbot.receive(
    "Problem: Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation.",
    user,
)


user (to chatbot):

Problem: Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation.

--------------------------------------------------------------------------------
chatbot (to user):

***** Suggested function Call: query_wolfram *****
Arguments: 
{
  "code": "solve (2x+10)(x+3)<(3x+9)(x+8) for x"
}
**************************************************

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED. USING AUTO REPLY FOR THE USER...
user (to chatbot):

***** Response from calling function "query_wolfram" *****
Assumption: solve (2 x + 10) (x + 3)<(3 x + 9) (x + 8) for x 
Answer: ans 0: x<-14
ans 1: x>-3

**********************************************************

--------------------------------------------------------------------------------
chatbot (to user):

The solution to the inequality $(2x+10)(x+3)<(3x+9)(x+8)$ is $x<-14$ or $x>-3$.

Expressed in interval no

## Calling a class method

We give an example of calling a class method. The class instance and the function name need to be passed to `UserProxyAgent` with field `class` and `func_name`.
```
user = UserProxyAgent(
    name="user",
    functions={
        "run_python_code": {
            "class": MathUserProxyAgent(use_docker=False), # a class instance
            "func_name": "_execute_one_python_code", # function to be called
        }
    }, 
    ...
)
```    

In [16]:
from flaml.autogen.agent import AssistantAgent, UserProxyAgent, MathUserProxyAgent

sys_prompt = """You are an advanced AI with the capability to solve complex math problems.
You can use a Python kernel as an external computing resource.

When the user gives a math problem, please use the most efficient way to solve the problem.
You are encouraged to use Python whenever it is possible during the solving process. For example, simplications, calculations, equation solving, etc.
However, if the operation requires little computation (very simple calculations, etc), you can also solve it directly.

Reply a single "TERMINATE" in the one message when everything is done.
"""
oai_config = {
    "functions": [
        {
            "name": "run_python_code",
            "description": "Return a tuple (str, bool): the execution result and if the execution is successful. The printed output or the errors will be returned.",
            "parameters": {
                "type": "object",
                "properties": {
                    "pycode": {
                        "type": "string",
                        "description": "The python code to be executed, with correct indentations.",
                    }
                },
                "required": ["pycode"],
            },
        }
    ],
    "function_call": "auto",
}

chatbot = AssistantAgent("chatbot", sys_prompt, config_list=config_list, **oai_config)

user = UserProxyAgent(
    "user",
    human_input_mode="NEVER",
    function_map={
        "run_python_code": {
            "class": MathUserProxyAgent(use_docker=False),
            "func_name": "_execute_one_python_code",
        }
    },
)

chatbot.receive(
    "Problem: Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation.",
    user,
)

user (to chatbot):

Problem: Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation.

--------------------------------------------------------------------------------
chatbot (to user):

The inequality is $(2x+10)(x+3)<(3x+9)(x+8)$. 

To solve this inequality, I will take the following steps:

1. Expand both sides of the inequality.
2. Bring all terms to one side to form a quadratic inequality.
3. Find the roots of the equivalent quadratic equation (by setting the inequality as an equation).
4. Map those roots onto a number line to determine the intervals for which the original inequality is true.

Let's carry out these steps using a python kernel.
***** Suggested function Call: run_python_code *****
Arguments: 
{
"pycode": "
from sympy import *

# declare the variable
x = symbols('x')

# define the inequality
ineq = (2*x+10)*(x+3)-(3*x+9)*(x+8)

# expand and simplify the inequality
ineq = simplify(ineq)

# solve the equality
roots